In [ ]:
# default_exp core

# Core Hierarchical 

> Module for Hierarchical Reconciliation.

In [ ]:
#export
from functools import partial
from inspect import signature
from typing import Callable, List

import numpy as np
import pandas as pd

In [ ]:
#hide
from fastcore.test import test_close

In [ ]:
#exporti
def _build_fn_name(fn, *args, inner_args) -> str:
    fn_name = fn.__name__
    func_params = signature(fn).parameters
    func_args = list(func_params.items())
    func_args = [(name, arg) for (name, arg) in func_args if arg.name not in inner_args]
    changed_kwargs = {
        name: value
        for value, (name, arg) in zip(args, func_args)
        if arg.default != value 
    }
    if changed_kwargs:
        changed_params = [f'{name}-{value}' for name, value in changed_kwargs.items()]
        fn_name += '_' + '_'.join(changed_params)
    return fn_name, changed_kwargs

In [ ]:
#exporti
def _as_tuple(x):
    if isinstance(x, tuple):
        return x
    return (x,)

In [ ]:
#export
class HierarchicalReconciliation:
    
    def __init__(self, reconcile_fns: List[Callable]):
        self.reconcile_fns = reconcile_fns
        
    def reconcile(self, Y_h: pd.DataFrame, Y_df: pd.DataFrame, S: pd.DataFrame):
        """Reconcile base forecasts.
        
            Parameters
            ----------
            Y_h: pd.DataFrame
                Base forecasts with columns ['ds'] 
                and models to reconcile indexed by 'unique_id'.
            Y_df: pd.DataFrame
                Training set of base time series with columns 
                ['ds', 'y'] indexed by 'unique_id'
                If a function of `self.reconcile_fns` receives
                residuals, `Y_df` must include them as columns.
            S: pd.DataFrame
                Summing matrix of size (hierarchies, bottom).
        """
        drop_cols = ['ds', 'y'] if 'y' in Y_h.columns else ['ds']
        model_names = Y_h.drop(columns=drop_cols, axis=1).columns.to_list()
        common_vals = dict(
            y = Y_df.pivot(columns='ds', values='y').loc[S.index].values,
            S = S.values,
            idx_bottom = [S.index.get_loc(col) for col in S.columns]
        )
        fcsts = Y_h.copy()
        for reconcile_fn_args in self.reconcile_fns:
            reconcile_fn, *args = _as_tuple(reconcile_fn_args)
            reconcile_fn_name, fn_kwargs = _build_fn_name(
                reconcile_fn, *args, 
                inner_args=['y', 'S', 'idx_bottom', 'y_hat', 'residuals']
            )
            has_res = 'residuals' in signature(reconcile_fn).parameters
            for model_name in model_names:
                y_hat_model = Y_h.pivot(columns='ds', values=model_name).loc[S.index].values
                if has_res:
                    common_vals['residuals'] = Y_df.pivot(columns='ds', values=model_name).loc[S.index].values.T
                kwargs = [key for key in signature(reconcile_fn).parameters if key in common_vals.keys()]
                kwargs = {key: common_vals[key] for key in kwargs}
                p_reconcile_fn = partial(reconcile_fn, y_hat=y_hat_model, **kwargs)
                fcsts_model = p_reconcile_fn(**fn_kwargs)
                fcsts[f'{model_name}/{reconcile_fn_name}'] = fcsts_model.flatten()
                if has_res:
                    del common_vals['residuals']
        return fcsts

In [ ]:
#hide
from hierarchicalforecast.methods import (
    bottom_up, top_down, 
    min_trace, empirical_risk_minimization
)
from hierarchicalforecast.utils import hierarchize

In [ ]:
#hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')
hiers_grouped = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]
hier_df, S = hierarchize(df, hiers_grouped)

In [ ]:
#hide
hier_df['y_model'] = hier_df['y']
# we should be able to recover y using the methods
hier_df_h = hier_df.groupby('unique_id').tail(12)
ds_h = hier_df_h['ds'].unique()
hier_df = hier_df.query('~(ds in @ds_h)')

In [ ]:
#hide
hrec = HierarchicalReconciliation(reconcile_fns=[
    #these methods should reconstruct the original y
    bottom_up, 
    (min_trace, 'ols'),
    (min_trace, 'wls_struct'),
    (min_trace, 'wls_var'),
    (min_trace, 'mint_shrink'),
])
reconciled = hrec.reconcile(hier_df_h, hier_df, S)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    test_close(reconciled['y'], reconciled[model])